In [1]:
import pandas as pd
import numpy as np
import sqlite3

from sklearn.model_selection import train_test_split


In [2]:
catalysts = pd.read_csv('./ref/final_cats.csv')
catalysts.head()


,ticker,disease,stage,date,catalyst,label
0,SPPI,Non-Hodgkin’s lymphoma,Approved,9/4/09,"Approved September 4, 2009.",5
1,JAZZ,Fibromyalgia,CRL,10/11/10,"CRL received October 11, 2010.",4
2,ASRT,Postherpetic neuralgia - shingles,Approved,1/28/11,"Approved January 28, 2011.",5
3,ASRT,Colorectal cancer,Approved,4/29/11,"Approved April 29, 2011.",5
4,SPPI,Colorectal cancer,Approved,4/29/11,"Approved April 29, 2011.",5


# Shifting catalyst Labels
Catalyst labels Map:
-1 -> 0 : BEARISH
0 -> 1: NEUTRAL
1 -> 2: BULLISH

In [4]:
catalysts['label'] = catalysts['label'].astype(int) + 1
features = catalysts.iloc[:, :-1]
labels = catalysts.iloc[:, -1:]

X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

print(f"Train Size = {X_train.shape[0]}")
print(f"Test Size = {X_test.shape[0]}")


Train Size = 3139
Test Size = 785


In [5]:
y_train.value_counts()


label
6        2128
5         638
4         373
Name: count, dtype: int64

In [6]:
train = pd.concat([X_train, y_train], axis=1).reset_index(drop=True)
test = pd.concat([X_test, y_test], axis=1).reset_index(drop=True)


In [7]:
train.head()


,ticker,disease,stage,date,catalyst,label
0,AZN,Neurofibromatosis type 1 plexiform neurofibromas,Approved,4/13/20,"FDA Approval announced April 13, 2020.",6
1,GOSS,Eosinophilic asthma,Phase 2b,10/13/20,Phase 2b trial did not meet primary endpoint -...,4
2,AZN,"Risk of cardiovascular (CV) death, hospitaliza...",Approved,5/9/23,"Approved May 9, 2023.",6
3,ABBV,Upper Limb Spasticity,Approved,7/29/21,"FDA approval announced July 29, 2021.",6
4,INFI,Triple negative breast cancer (TNBC) and ovari...,Phase 1,12/9/20,Phase 1 initial data due presented at SABCS An...,5


In [8]:
test.head()


,ticker,disease,stage,date,catalyst,label
0,RLAY,Cholangiocarcinoma (CCA),Phase 1,10/12/23,Phase 1 data from non-CCA expansion cohort pre...,6
1,GSK,Rheumatoid Arthritis,Phase 2b,10/21/18,"Phase 2b released October 21, 2018 did not mee...",4
2,ABBV,Episodic Migraine,Phase 3,4/21/23,Phase 3 trial met its primary and secondary en...,5
3,CRSP,Advanced clear cell renal cell carcinoma,Phase 1,11/10/22,Phase 1 result reported a 77% disease control ...,6
4,RHHBY,Follicular lymphoma,Phase 3,6/10/22,Phase 3 final analysis reported that PFS was s...,6


In [9]:
train.to_csv("./ref/train.csv", index=False)
test.to_csv("./ref/test.csv", index=False)


In [10]:
train.shape


(3139, 6)

In [11]:
test.shape


(785, 6)